In [1]:
import os
import sys

from pathlib import Path

# Get the current working directory
cwd = Path(os.getcwd())

# Assuming your notebook is in the project root, set the project root as cwd
project_root = cwd.parent
print(project_root)

# If your notebook is in a subdirectory of the project root, you can modify the path accordingly:
# project_root = cwd.parent  # Go up one directory level
# project_root = cwd.parent.parent  # Go up two directory levels

# Add the project root directory to sys.path
sys.path.insert(0, str(project_root))
# sys.path.append('../')

import warnings
import corr_fitter.bs_utils as bs
import corr_fitter.bs_analysis as bs_analysis
import corr_fitter.load_data_priors as ld
import corr_fitter.corr_fit_analysis as fa
import corr_fitter.corr_fitter as fitter
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib
import h5py as h5
import importlib
import numpy as np
import gvar as gv
import pandas as pd
import platform
from tabulate import tabulate


warnings.simplefilter(action="default")
warnings.filterwarnings('ignore')


/Users/grantdb/lqcd/corr_fitter


In [2]:
# Define paths and other variables
if platform.system() == 'Darwin':
    base_dir = '/Users/grantdb/lqcd/corr_fitter'
else:
    base_dir = '/home/gmoney/lqcd/corr_fitter'
input_dir = os.path.join(base_dir, "tests/input_files")
data_dir = os.path.join(base_dir, "data")
fit_results_dir = os.path.join(base_dir, "fit_results")
if platform.system() == 'Darwin':
    data_file = '/Users/grantdb/lqcd/data/c51_2pt_octet_decuplet.h5'
else:
    data_file = '/home/gmoney/lqcd/data/c51_2pt_octet_decuplet.h5'
# hyperon_data_file = os.path.join(data_dir,"hyperon_data.h5")
bs_data_file = os.path.join(data_dir, "hyperon_bs_data.h5")
model_type = 'hyperons'
t_plot_min = 0
t_plot_max = 40
bs_N = 100
bs_seed = "seed"

In [13]:
importlib.reload(fa)
importlib.reload(fitter)
abbr = 'a09m260'
fit_params = os.path.join(input_dir, f"{abbr}.py")
pdf_name = 'output_stability'+abbr +'.pdf'
sys.path.append(os.path.dirname(os.path.abspath(fit_params)))
fp = importlib.import_module(fit_params.split('/')[-1].split('.py')[0])
p_dict = fp.p_dict
prior = fp.prior
nstates = p_dict['n_states']
corrs = ld.get_corrs(data_file=data_file,p_dict=p_dict)
proton = ld.get_raw_corr(data_file, p_dict['abbr'], particle='proton')
proton_ = gv.dataset.avg_data(proton)
# print(proton)
fit = fitter.Fitter(n_states=nstates,prior=prior,t_range=p_dict['t_range'],states=['proton'],p_dict = p_dict, raw_corrs=proton_,model_type='hyperons',simult=False)
# print(fit.get_fit())
fit_analysis = fa.Fitter_Analysis(t_range=p_dict['t_range'], simult=False,
                               states=['proton'], p_dict=p_dict, n_states=nstates,
                               prior=prior, corr_gv=proton, model_type=model_type)
print(fit_analysis.get_fit())
# fit_analysis.plot_stability(pdf_name, t_start=5, t_end=25,
                            # model_type=model_type, n_states_array=[1,2,3,4]
                            # )

        
hyperon_fit = fa.analyze_hyperon_corrs(data_file, fit_params,proton,show_eff=False,show_stab=True, t_start=5,t_end=10,pdf_name=pdf_name,
                                            model_type=model_type,bs=False, bs_file=bs_data_file,bs_path=abbr, bs_N=bs_N, bs_seed=bs_seed)
print(hyperon_fit)


proton
proton
Least Square Fit:
  chi2/dof [dof] = 1.6 [40]    Q = 0.014    logGBF = 1032.1

Parameters:
   proton_z_PS 0      3.575(49)e-07        [   0.0000 (33) ]  
               1      1.657(43)e-06        [   0.0000 (33) ]  
   proton_z_SS 0      1.032(12)e-07        [  4.4(4.4)e-06 ]  
               1      2.649(61)e-07        [  4.4(4.4)e-06 ]  
       delta_E 0          0.6 (3.3)        [     0.6 (3.3) ]  
               1          0.7 (3.2)        [     0.7 (3.2) ]  
    delta_z_PS 0    5e-24 +- 0.0033        [   0.0000 (33) ]  
               1   -8e-24 +- 0.0033        [   0.0000 (33) ]  
    delta_z_SS 0       4.4(4.4)e-06        [  4.4(4.4)e-06 ]  
               1       4.4(4.4)e-06        [  4.4(4.4)e-06 ]  
       sigma_E 0          0.5 (3.0)        [     0.5 (3.0) ]  
               1          0.7 (3.2)        [     0.7 (3.2) ]  
    sigma_st_E 0          0.50 (30)        [     0.50 (30) ]  
               1          0.6 (3.2)        [     0.6 (3.2) ]  
 sigma_st_z_P

TypeError: float() argument must be a string or a real number, not 'gvar._gvarcore.GVar'

In [4]:
ld.generate_latex_line(hyperon_fit=hyperon_fit[4])

NameError: name 'hyperon_fit' is not defined

In [ ]:
# from tabulate import tabulate

# abbreviations = [f[:-3] for f in os.listdir(input_dir) if f.endswith('.py')]

# for abbr in abbreviations:
#     fit_params = os.path.join(input_dir, f"{abbr}.py")
#     if not os.path.exists(fit_params):
#         print(f"Error: input file {fit_params} does not exist!")
#         continue

#     with open(fit_params, 'r') as f:
#         input_file_contents = f.read()

#     if 'p_dict' not in input_file_contents:
#         print(f"Error: input file {fit_params} does not contain a dictionary called 'p_dict'!")
#         continue

#     try:
#         p_dict = {}
#         exec(input_file_contents, p_dict)
#     except Exception as e:
#         print(f"Error: Failed to execute the contents of input file {fit_params}!\n{str(e)}")
#         continue

#     if 'tag' not in p_dict:
#         print(f"Warning: input file {fit_params} does not contain a dictionary called 'tag' within the 'p_dict' dictionary! Adding default values...")
#         p_dict['tag'] = {
#             'sigma' : 'sigma',
#             'sigma_st' : 'sigma_st',
#             'xi' :  'xi',
#             'xi_st' : 'xi_st',
#             'lam' : 'lam',
#         }

#     try:
#         hyperon_fit = fa.analyze_hyperon_corrs(data_file, fit_params, model_type=model_type,
#                                                bs=False, bs_file=bs_data_file,
#                                                bs_path=abbr, bs_N=bs_N, bs_seed=bs_seed)
#     except KeyError:
#         print(f"KeyError: Error analyzing hyperons for input file {fit_params}. Skipping abbreviation {abbr}.")
#         continue

#     my_fit = hyperon_fit.get_fit()

#     out_path = os.path.join(fit_results_dir, abbr, model_type)
#     ld.pickle_out(fit_out=my_fit, out_path=out_path, species="hyperons")
#     plot1 = hyperon_fit.return_best_fit_info()
#     plot2 = hyperon_fit.plot_effective_mass(t_plot_min=t_plot_min, t_plot_max=t_plot_max, model_type=model_type,
#                                             show_plot=True, show_fit=True)

#     output_dir = os.path.join(fit_results_dir, abbr, f"{model_type}_{abbr}")
#     os.makedirs(output_dir, exist_ok=True)
#     output_pdf = os.path.join(output_dir, 'output.pdf')
#     with PdfPages(output_pdf) as pp:
#         pp.savefig(plot1)
#         pp.savefig(plot2)

#     params_df = pd.DataFrame(my_fit.p).transpose()

#     print("Abbreviation:", abbr)
#     print(tabulate(params_df, headers='keys', tablefmt='fancy_grid'))

#     # Ask the user if the fit result is acceptable
#     while True:
#         result = input(f"Is the fit result for abbreviation {abbr} acceptable? (y/n): ")
#         if result == "y":
#             # Save the result and skip this fit in future runs
#             out_path = 'fit_results/{0}/{1}/'.format(abbr, model_type)
#             ld.pickle_out(fit_out=my_fit, out_path=out_path, species="hyperons")
#             break
#         elif result == "n":
#             break
